In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import networkx as nx
import json
import sys
import time
from datetime import datetime
import operator
import pandas as pd
import xlrd

In [ ]:
CC_mapper={}
class Review:
    def __init__(self,userid,useridmapped,prodid,prodidmapped,rating,label,date):
        self.userid=userid
        self.useridmapped=useridmapped
        self.prodid=prodid
        self.prodidmapped=prodidmapped
        self.rating=rating
        self.label=label
        self.date=date

    def __repr__(self):
        return '{}'.format(self.userid)
    
    def __hash__(self):
        return hash((self.userid))

    def __eq__(self, other):
        return self.userid== other.userid

    def __ne__(self, other):
        return not self.__eq__(other)
    
   
def isAdjacent(e1,e2):
    if e1[0]==e2[0] or e1[1]==e2[0] or e1[0]==e2[1] or e1[1]==e2[1]:
        return True
    return False

def degree(G,edge):
    return G.degree[edge[0]]+G.degree[edge[1]]-1

def canbeincluded(userset):
    if len(userset)==0:
        return 0
    union=set()
    intersection=allusers[list(userset)[0]]
    for u in userset:
        union=union.union(allusers[u])
        intersection=intersection.intersection(allusers[u])
        jaccard=len(intersection)/len(union)*1.0
        if jaccard>0.5:
            return 1
        return 0

In [ ]:
minn={}
d={}

wb=xlrd.open_workbook("/content/drive/My Drive/Yelp Metadata.xlsx")
filee = wb.sheet_by_index(0) 

In [ ]:
#code for finding the date at which the first review of the product was made 


for i in range (1,int(filee.nrows/8)):
    
    fsplit=filee.row_values(i)
    
    userid=fsplit[0]
    prodid=int(fsplit[1])
    rating=int(round(float(fsplit[2])))  
   
    

    date=fsplit[3]
    if prodid not in d:
        minn[prodid]=0
        d[prodid]=datetime(*xlrd.xldate_as_tuple(date, wb.datemode)).date()
        
      
    minn[prodid]=datetime(*xlrd.xldate_as_tuple(date, wb.datemode)).date()
    if minn[prodid]<d[prodid]:
        d[prodid]=minn[prodid]       

        


In [ ]:
#code for finding the days difference b/w first review and current date 
#code for creating list of reviewers corresponding to each product-rating
#code for adding nodes in the graph
G = nx.Graph()
reviewsperproddata={} 
nodedetails={}
prodlist={} #list of reviews corresponding to each product-rating
dictprod={} #node number corr. to each prod.-rating
dictprodr={} #node number corresponding to each product-rating
count=0


for i in range (1,int(filee.nrows/8)):
    
    fsplit=filee.row_values(i)
    
    userid=int(fsplit[0])      
    prodid=int(fsplit[1])
    rating=str(int(round(float(fsplit[2]))))
    date=fsplit[3]
    newdate=datetime(*xlrd.xldate_as_tuple(date, wb.datemode)).date()
    datetodays=(newdate-d[prodid]).days

    review=Review(userid,'',prodid,'',rating,label,datetodays)
    
  
    k=str(prodid)+"_"+str(rating)
    if k not in reviewsperproddata:
        count=count+1
        reviewsperproddata[k]=set()
        dictprod[count]=k
        dictprodr[k]=count
        prodlist[k]=[]
        G.add_node(count)

    prodlist[k].append(review)

    reviewsperproddata[k].add(review)
    


    

In [ ]:
#code for finding the list of reviewers who co-reviewed the product within given time diff.


graphlist=list(G.nodes())
cr={}
for node1i in range(len(graphlist)):
    node1=graphlist[node1i]
    if node1 not in cr:
        cr[node1]=[]
    for u1i in range(len(prodlist[dictprod[node1]])): 
        u1=prodlist[dictprod[node1]][u1i] 
        cr11=set()     
        cr11.add(u1)     
        for u2i in range(u1i+1,len(prodlist[dictprod[node1]])):
            u2=prodlist[dictprod[node1]][u2i]
            if abs(u1.date-u2.date)<10:
                cr11.add(u2)
        cr[node1].append(cr11)  
    cr[node1].sort(key=len,reverse=True)





In [ ]:
#in this code, finding edge attributes and hence, graph is constructed also finding set of common reviewer b/w edges
edgedetails={}
cmnrevrsedges={}
cmnrevrslist={} # list of common reviewers at each node number
cmnrevrslistr={} #reverse list of cmnrevrslist
cmnrevrsedgeslen={}
countt=0
visited={}
edgecount={}
for node1i in range(len(graphlist)):

    node1=graphlist[node1i]
    for node2i in range(node1i+1,len(graphlist)):
        node2=graphlist[node2i]

        maxx=0
        maxxcr=set()
        
        cr1=cr[node1]
        cr2=cr[node2]   
        crlist=set()
        f=0
        for cri1 in cr1:
            if len(cri1)<2:
                break
            for cri2 in cr2:
                if len(cri2)<2:
                    f=1
                    break
                crr=cri1.intersection(cri2)
                crr=frozenset(crr)
                if len(crr)>1:
                    crlist.add(crr)

            if f==1:
                break
        
        crlist=list(crlist)
        crlist.sort(key=len,reverse=True)

        for commonreviewers in crlist:
            if len(commonreviewers)>1: #not considering common single reviwer between products(not considering single reviewer group)
                
                if commonreviewers not in cmnrevrslistr:
                    countt=countt+1
                    cmnrevrslist[countt]=commonreviewers
                    cmnrevrslistr[commonreviewers]=countt
                    maincount=countt
                else:
                    maincount=cmnrevrslistr[commonreviewers]
                if node1<node2:
                    n1=node1
                    n2=node2
                else:
                    n1=node2
                    n2=node1

                if maincount not in cmnrevrsedges:
                    cmnrevrsedges[maincount]=[]
               
                
                    
                if (n1,n2) not in edgecount: 
                     edgecount[(n1,n2)]=0
                     G.add_edge(n1,n2)
                     edgedetails[(n1,n2)]=crlist

                if (n1,n2) not in cmnrevrsedges[maincount]:
                    cmnrevrsedges[maincount].append((n1,n2))
                    edgecount[(n1,n2)]=edgecount[(n1,n2)]+1
              
              


In [ ]:

#code for adding reviewer groups at isolated nodes into final group set


for node in G.nodes():
    if G.degree[node]==0:
        
        k=frozenset(reviewsperproddata[dictprod[node]])
        
        if k not in CC_mapper:
            CC_mapper[k]=str(dictprod[node])
        #else:
        #    CC_mapper[k]=CC_mapper[k]+':'+str(dictprod[node])
            
#print(CC_mapper) #list of reviewers at isolated node with the target product and rating






In [ ]:
poppinglist=[]
for item in cmnrevrsedges:
    print(cmnrevrsedges[item])
    if len(cmnrevrsedges[item])==1:
         
         poppinglist.append(item)

for p in poppinglist:
    cmnrevrsedges.pop(p)

    




[(17, 33)]
[(119, 258)]
[(182, 317)]
[(204, 332), (204, 481)]
[(316, 410)]
[(357, 480)]
[(357, 480)]
[(357, 480)]
[(357, 480)]
[(357, 480)]
[(357, 480)]
[(357, 480)]
[(357, 480)]
[(480, 500)]


In [ ]:
for c in cmnrevrsedges:
    if c not in cmnrevrsedgeslen:
       
        cmnrevrsedgeslen[c]=0
    cmnrevrsedgeslen[c]=len(cmnrevrsedges[c])
    

sorted_cmnrevrsedgeslen=sorted(cmnrevrsedgeslen.items(), key=operator.itemgetter(1))


cmnrevrsedgeslen=sorted_cmnrevrsedgeslen

#print(cmnrevrsedgeslen)
#print(len(cmnrevrsedgeslen))

In [ ]:
import matplotlib.pyplot as plt
#print(len(G.edges()))
#print(len(G.nodes()))


#print(edgedetails)




In [ ]:
#print(list(G.edges()))
#print(list(G.nodes()))

In [ ]:

print(cmnrevrsedgeslen)

for ci in range(len(cmnrevrsedgeslen)):

    userset=set()
    prodset=set()
    f=0
    i=set(cmnrevrslist[cmnrevrsedgeslen[ci][0]])
    
    for cj in range(ci+1,len(cmnrevrsedgeslen)):
        j=set(cmnrevrslist[cmnrevrsedgeslen[cj][0]])
        if i.difference(j)==0:
            if canbeincluded(j):
                
                userset=userset.union(i.union(j))
                
                for edge in cmnrevrsedges[cmnrevrsedgeslen[cj][0]]:
                    
                    if cmnrevrslist[cmnrevrsedgeslen[cj][0]] in edgedetails[(edge[0],edge[1])]:
                        
                        edgecount[(edge[0],edge[1])]=edgecount[(edge[0],edge[1])]-1
                        edgedetails[(edge[0],edge[1])].remove(cmnrevrslist[cmnrevrsedgeslen[cj][0]])
                        if edgecount[(edge[0],edge[1])]==0:
                            G.remove_edge(edge[0],edge[1])
                        f=1
            else:
                uset=set()
                pset=set()
                ss=''
                k=j.difference(i)
                
                if canbeincluded(k):
                    uset=k
                    k=frozenset(k)
                    pset=allusers[list(uset)[0]]
                    for u in uset:
                        pset=pset.intersection(allusers[u])
                    
                    for p in pset:
                        ss=ss+str(p)+'_0'+':'

                    ss=ss[0:len(ss)-1]
                    if k not in CC_mapper:
                        CC_mapper[k]=str(ss)
                    else:
                        CC_mapper[k]=CC_mapper[k]+':'+ss
                    f=1
            
            if f==1:
                for edge in cmnrevrsedges[cmnrevrsedgeslen[ci][0]]:   
                        if cmnrevrslist[cmnrevrsedgeslen[ci][0]] in edgedetails[(edge[0],edge[1])]:
                            edgecount[(edge[0],edge[1])]=edgecount[(edge[0],edge[1])]-1
                            edgedetails[(edge[0],edge[1])].remove(cmnrevrslist[cmnrevrsedgeslen[ci][0]])
                            if edgecount[(edge[0],edge[1])]==0:
                                G.remove_edge(edge[0],edge[1])


    ss=''
    if len(userset)>0:
        prodset=allusers[list(userset)[0]]
        for u in userset:
            prodset=prodset.intersection(allusers[u])
        for p in prodset:
            ss=ss+str(p)+'_0'+':'

        ss=ss[0:len(ss)-1]  
        k=frozenset(userset)
        if k not in CC_mapper:
            CC_mapper[k]=str(ss)
        else:
            CC_mapper[k]=CC_mapper[k]+':'+ss

print(len(CC_mapper))

print(len(G.nodes()))
print(len(G.edges()))

[(4, 2)]
492
506
8


In [ ]:
#print(cmnrevrsedges)

In [ ]:
#print(list(G.nodes()))
#print(list(G.edges()))

In [ ]:
t=set(CC_mapper)
GG=nx.Graph()      
for cmnrvr in cmnrevrsedges:
    nodes=[]
    ss=''
    for edge in cmnrevrsedges[cmnrvr]:
        if edge[0] not in  nodes:
            nodes.append(edge[0])
            ss=ss+dictprod[edge[0]]+":"

        if edge[1] not in  nodes:
            nodes.append(edge[1])
            ss=ss+dictprod[edge[1]]+":"
        edgecount[(edge[0],edge[1])]=edgecount[(edge[0],edge[1])]-1
        edgedetails[(edge[0],edge[1])].remove(cmnrevrslist[cmnrvr])
        if edgecount[(edge[0],edge[1])]==0:
            G.remove_edge(edge[0],edge[1])
    ss=ss[0:len(ss)-1]  
    k=frozenset(cmnrevrslist[cmnrvr])
    if k not in CC_mapper:
        CC_mapper[k]=str(ss)
    else:
        CC_mapper[k]=CC_mapper[k]+':'+ss


'''        
print(len(CC_mapper))
p=set(CC_mapper)
print(p.difference(t))


print(list(G.edges()))
print(list(G.nodes()))
'''

'        \nprint(len(CC_mapper))\np=set(CC_mapper)\nprint(p.difference(t))\n\n\nprint(list(G.edges()))\nprint(list(G.nodes()))\n'

In [ ]:
import matplotlib.pyplot as plt
nx.write_gexf(G,"Graph.gexf")


In [ ]:
print(len(G.edges()))

6


In [ ]:





co=0
while len(G.edges())>0:
    
    
    
    
    
    co=co+1
    print (str(co)+"\t"+str(len(G.edges())))

    cmnrevrsedges2={}
    cmnrevrslist2={}
    cmnrevrslistr2={}
    countt2=0
    visited={}
    edgedetails2={}
    edgecount2={}
    edgelist=list(G.edges())
    for edge1i in range(len(edgelist)):
        edge1=edgelist[edge1i]

        if edge1[0]<edge1[1]:
            e10=edge1[0]
            e11=edge1[1]
        else:
            e10=edge1[1]
            e11=edge1[0]
            
        s1=str(dictprod[e10])+":"+str(dictprod[e11])

        if edge1i==0:
            count=count+1
            node1=count
            dictprod[count]=s1
            dictprodr[s1]=count
            GG.add_node(count)
            reviewsperproddata[s1]=edgedetails[(e10,e11)]
        else:
            node1=dictprodr[s1]
        
        for edge2i in range(edge1i+1,len(edgelist)):
            edge2=edgelist[edge2i]

            if edge2[0]<edge2[1]:
                e20=edge2[0]
                e21=edge2[1]
            else:
                e20=edge2[1]
                e21=edge2[0]
                
            s2=str(dictprod[e20])+":"+str(dictprod[e21])

            if edge1i==0:
                count=count+1
                node2=count
                dictprod[count]=s2
                dictprodr[s2]=count
                GG.add_node(count)
                reviewsperproddata[s2]=edgedetails[(e20,e21)]
            else:
                node2=dictprodr[s2]

            if isAdjacent(edge1,edge2):
                cr1=set(reviewsperproddata[dictprod[node1]])
                cr2=set(reviewsperproddata[dictprod[node2]])
                
                crlist=set()
                f=0
                for cri1 in cr1:
                    if len(cri1)<2:
                        break
                    for cri2 in cr2:
                        if len(cri2)<2:
                            f=1
                            break
                        crr=cri1.intersection(cri2)
                        crr=frozenset(crr)
                        if len(crr)>1:
                            crlist.add(crr)
                    if f==1:
                        break
                
                crlist=list(crlist)
                crlist.sort(key=len,reverse=True)

                for commonreviewers in crlist:
                        
                    if len(commonreviewers)>1 and commonreviewers not in cmnrevrslistr:
                        if commonreviewers not in cmnrevrslistr2:
                            countt2=countt2+1
                            cmnrevrslist2[countt2]=commonreviewers
                            cmnrevrslistr2[commonreviewers]=countt2
                            maincount=countt2
                        else:
                            maincount=cmnrevrslistr2[commonreviewers]

                        if maincount not in cmnrevrsedges2:
                                cmnrevrsedges2[maincount]=[]

                      

                        if (node1,node2) not in edgecount2:
                                GG.add_edge(node1,node2)
                                edgecount2[(node1,node2)]=0
                                edgedetails2[(node1,node2)]=crlist
                        if (node1,node2) not in cmnrevrsedges2[maincount]:                              
                                cmnrevrsedges2[maincount].append((node1,node2))
                                edgecount2[(node1,node2)]=edgecount2[(node1,node2)]+1
                        


    for node in GG.nodes():
        if GG.degree[node]==0:

            k=reviewsperproddata[dictprod[node]][0]
            if k not in CC_mapper:
                CC_mapper[k]=str(dictprod[node])
            else:
                CC_mapper[k]=CC_mapper[k]+':'+str(dictprod[node])

    G=GG
    cmnrevrsedges=cmnrevrsedges2
    cmnrevrslist=cmnrevrslist2
    cmnrevrslistr=cmnrevrslistr2
    edgedetails=edgedetails2
    edgecount=edgecount2
    cmnrevrsedgeslen={}
    mark={}
    
    poppinglist=[]
    for item in cmnrevrsedges:
        if len(cmnrevrsedges[item])==1:
             poppinglist.append(item)

    for p in poppinglist:
        cmnrevrsedges.pop(p)
    
    for c in cmnrevrsedges:
        if c not in cmnrevrsedgeslen:
            cmnrevrsedgeslen[c]=0
            mark[c]=0
        cmnrevrsedgeslen[c]=len(cmnrevrsedges[c])
    sorted_cmnrevrsedgeslen=sorted(cmnrevrsedgeslen.items(), key=operator.itemgetter(1))
    cmnrevrsedgeslen=sorted_cmnrevrsedgeslen

    for ci in range(len(cmnrevrsedgeslen)):
        userset=set()
        prodset=set()
        f=0
        i=set(cmnrevrslist[cmnrevrsedgeslen[ci][0]])
        for cj in range(ci+1,len(cmnrevrsedgeslen)):
            j=set(cmnrevrslist[cmnrevrsedgeslen[cj][0]])
            if i.difference(j)==0:
                if canbeincluded(j):
                    mark[cmnrevrsedgeslen[ci][0]]=1
                    mark[cmnrevrsedgeslen[cj][0]]=1
                    userset=userset.union(i.union(j))
                    for edge in cmnrevrsedges[cmnrevrsedgeslen[cj][0]]:
                       
                        if cmnrevrslist[cmnrevrsedgeslen[cj][0]] in edgedetails[(edge[0],edge[1])]:
                            edgecount[(edge[0],edge[1])]=edgecount[(edge[0],edge[1])]-1
                            edgedetails[(edge[0],edge[1])].remove(cmnrevrslist[cmnrevrsedgeslen[cj][0]])
                            if edgecount[(edge[0],edge[1])]==0:
                                G.remove_edge(edge[0],edge[1])
                            f=1
                else:
                    uset=set()
                    pset=set()
                    ss=''
                    k=j.difference(i)
                    if canbeincluded(k):
                        uset=k
                        k=frozenset(k)
                        pset=allusers[list(uset)[0]]
                        for u in uset:
                            pset=pset.intersection(allusers[u])
                        
                        for p in pset:
                            ss=ss+str(p)+'_0'+':'

                        ss=ss[0:len(ss)-1]
                        if k not in CC_mapper:
                            CC_mapper[k]=str(ss)
                        else:
                            CC_mapper[k]=CC_mapper[k]+':'+ss
                        f=1
                if f==1:
                    for edge in cmnrevrsedges[cmnrevrsedgeslen[ci][0]]:   
                        if cmnrevrslist[cmnrevrsedgeslen[ci][0]] in edgedetails[(edge[0],edge[1])]:
                            edgecount[(edge[0],edge[1])]=edgecount[(edge[0],edge[1])]-1
                            edgedetails[(edge[0],edge[1])].remove(cmnrevrslist[cmnrevrsedgeslen[ci][0]])
                            if edgecount[(edge[0],edge[1])]==0:
                                G.remove_edge(edge[0],edge[1])

        ss=''
        if len(userset)>0:
            prodset=allusers[list(userset)[0]]
            for u in userset:
                prodset=prodset.intersection(allusers[u])
            for p in prodset:
                ss=ss+str(p)+'_0'+':'

            ss=ss[0:len(ss)-1]  
            k=frozenset(userset)
            if k not in CC_mapper:
                CC_mapper[k]=str(ss)
            else:
                CC_mapper[k]=CC_mapper[k]+':'+ss

    GG=nx.Graph()

    for cmnrvr in cmnrevrsedges:
        nodes=[]
        ss=''
        for edge in cmnrevrsedges[cmnrvr]:
            if edge[0] not in  nodes:
                nodes.append(edge[0])
                ss=ss+dictprod[edge[0]]+":"

            if edge[1] not in  nodes:
                nodes.append(edge[1])
                ss=ss+dictprod[edge[1]]+":"
            edgecount[(edge[0],edge[1])]=edgecount[(edge[0],edge[1])]-1
            edgedetails[(edge[0],edge[1])].remove(cmnrevrslist[cmnrvr])
            if edgecount[(edge[0],edge[1])]==0:
                G.remove_edge(edge[0],edge[1])
        ss=ss[0:len(ss)-1]  
        k=frozenset(cmnrevrslist[cmnrvr])
        if k not in CC_mapper:
            CC_mapper[k]=str(ss)
        else:
            CC_mapper[k]=CC_mapper[k]+':'+str(ss)

grps={}
co=0
for us in CC_mapper:
    c=0
    denom=0
    userset=set()
    prodset=set()
    for u in us:
        userset.add(int(u.userid))
    prods=CC_mapper[us].split(':')
    
    for p in prods:
        s=p.find('_')
        
        prodset.add(int(p[:s]))
        

    #print(prodset)

    if len(grps) not in grps and len(prodset)>0 and len(userset)>1:
        grps[len(grps)]={'id':len(grps),'users':list(userset),'prods':list(prodset),'scorepred':0, 'scoregt':0, 'fakegt':0,'fakepred':0}
 

1	6


In [ ]:
with open("/content/drive/My Drive/convertcsv.json", 'w') as fp:
    json.dump(grps, fp) 

print ('end')            


end


In [ ]:
import networkx as nx
import xlrd
import json
import sys
import time
import ast
import csv
import random
import numpy as np
from datetime import datetime
import math
from sklearn.cluster import KMeans
import operator
import pandas as pd
import re
from collections import Counter

In [ ]:
class Groups:
    def __init__(self,users,prods):
        self.users=users
        self.prods=prods
    def __lt__(self, other):
        return len(self.users) < len(other.users)

class Review:
    def __init__(self,userid,useridmapped,prodid,prodidmapped,rating,label,date,content):
        self.userid=userid
        self.useridmapped=useridmapped
        self.prodid=prodid
        self.prodidmapped=prodidmapped
        self.rating=rating
        self.label=label
        self.date=date
        self.content=content

    def __repr__(self):
        return '({})'.format(self.prodid)
    
    def __hash__(self):
        return hash(self.prodid)

    def __eq__(self, other):
        return self.prodid== other.prodid

    def __ne__(self, other):
        return not self.__eq__(other) 

In [ ]:
text=[]
wb=xlrd.open_workbook("/content/drive/My Drive/Yelp dataset.xlsx")
filee = wb.sheet_by_index(0)
for i in range (1,int(filee.nrows/8)):
    
    fsplit=filee.row_values(i)

    text.append(fsplit[3].strip())



In [ ]:
allprods={}
allusers={}
reviewtime={}
reviewrating={}
reviewcontent={}
wholerev={}
minn={}
d={}
fake=set()
rvdate={}
maxrvdate={}
maxrvcon={}
wb=xlrd.open_workbook("/content/drive/My Drive/Yelp Metadata.xlsx")
filee = wb.sheet_by_index(0)
c=0
for i in range (1,int(filee.nrows/8)):
    
    fsplit=filee.row_values(i)
    
    userid=int(fsplit[0])
    prodid=int(fsplit[1])
    rating=int(round(float(fsplit[2])))  
  
    date=fsplit[3]
    
    date=datetime(*xlrd.xldate_as_tuple(date, wb.datemode)).date()
    if prodid not in d:
        minn[prodid]=0
        d[prodid]=date
      
    minn[prodid]=date
    if minn[prodid]<d[prodid]:
        d[prodid]=minn[prodid]

    if userid not in rvdate:
        rvdate[userid]={}
        maxrvdate[userid]={}
        maxrvcon[userid]={}
    if prodid not in rvdate[userid]:
        rvdate[userid][prodid]=date
        maxrvdate[userid][prodid]=date
        maxrvcon[userid][prodid]=text[c]

    rvdate[userid][prodid]=date
    if rvdate[userid][prodid]>maxrvdate[userid][prodid]:
        maxrvdate[userid][prodid]=rvdate[userid][prodid]
        maxrvcon[userid][prodid]=text[c]

    c=c+1


In [ ]:

for i in range (1,int(filee.nrows/8)):
    
    fsplit=filee.row_values(i)
    
    userid=int(fsplit[0])
    prodid=int(fsplit[1])
    rating=int(round(float(fsplit[2])))  
    date=fsplit[3]
    
    newdate=datetime(*xlrd.xldate_as_tuple(date, wb.datemode)).date()
    if (newdate==maxrvdate[userid][prodid]):
        

        datetodays=(newdate-d[prodid]).days

        r = Review(userid,'',prodid,'',rating,label,datetodays,maxrvcon[userid][prodid] )

        if userid not in reviewtime:
            reviewtime[userid]={}
        if prodid not in reviewtime[userid]:
            reviewtime[userid][prodid]=datetodays#this user reviewed this product this time
        if userid not in reviewrating:
            reviewrating[userid]={}
        if prodid not in reviewrating[userid]:
            reviewrating[userid][prodid]=rating#this user gave this product this rating
        if userid not in reviewcontent:
            reviewcontent[userid]={}
        if prodid not in reviewcontent[userid]:
            reviewcontent[userid][prodid]=maxrvcon[userid][prodid] #this user gave this product this review
        if userid not in allusers:
            allusers[userid]=[]        
        if prodid not in allprods:
            allprods[prodid]=[]
        if userid not in wholerev:
            wholerev[userid]={}
        if prodid not in wholerev[userid]:
            wholerev[userid][prodid]=r#storing review type object 
       

        allprods[prodid].append(userid) #list of users reviewed this product
        allusers[userid].append(prodid)  #list of products reviewed by this user

        c=c+1
        
        

In [ ]:
def reviewtightness(group, L):
    v = 0
    for user in group.users:
        for prod in group.prods:
            # prod=prod.split("_")[0]
            if prod in reviewtime[user]:
                v = v + 1
    if len(group.prods) == 0:
        return 0
    return (v * L) / (1.0 * len(group.users) * len(group.prods))


def neighbortightness(group, L):

    userlist = list(group.users)
    denom = 0
    num = 0
    for user1i in range(len(userlist)):
        user1 = userlist[user1i]
        for user2i in range(user1i + 1, len(userlist)):
            user2 = userlist[user2i]
            union = set(allusers[user1]).union(set(allusers[user2]))
            intersection = set(allusers[user1]).intersection(
                set(allusers[user2]))
            num = num + len(intersection) / (len(union) * 1.0)
            denom = denom + 1

    return (num * L) / (1.0 * denom)


def producttightness(group):

    c = 0
    userlist = list(group.users)
    for user in userlist:
        if c == 0:
            intersection = set(allusers[user])
            union = set(allusers[user])
        else:
            intersection = intersection.intersection(set(allusers[user]))
            union = union.union(set(allusers[user]))
        c = c + 1

    return len(intersection) / (len(union) * 1.0)


def averagetimewindow_ratingvariance(group, L):

    if len(group.prods) == 0:
        return 0, 0
    avg = 0
    var = 0
    for prod in group.prods:
        prodlist = []
        prodtym = []
        # prod=prod.split("_")[0]
        minn = float('inf')
        maxx = 0
        for user in group.users:
            if prod in reviewtime[user]:
                prodlist.append(reviewrating[user][prod])
                prodtym.append(reviewtime[user][prod])

        var = var + np.var(prodlist)
        ans = np.std(prodtym)
        if ans < 30:
            avg = avg + (1 - ans / 30.0)

    var = var / (-1.0 * len(group.prods))
    rating_variance = 2 * (1 - (1.0 / (1 + math.exp(var))))

    return (avg * L) / (1.0 * len(group.prods)), rating_variance * L


def productreviewerratio(group):

    maxx = 0

    for prod in group.prods:

        num = 0
        denom = 0
        for user in group.users:
            if prod in reviewtime[user]:
                num = num + 1

        for r in allprods[prod]:
            # if int(r.rating)==int(prod.split("_")[1]):
            denom = denom + 1

        ans = num / (1.0 * denom)
        if ans > maxx:
            maxx = ans

    return maxx


def groupsize(group):
    return 1 / (1 + math.exp(3 - len(group.users)))


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def cosine(content_a, content_b):

    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result


def GCS(group):

    maxx = 0
    for prod in group.prods:
        avg = 0
        c = 0
        userlist = list(group.users)
        for r1i in range(len(userlist)):
            r1 = userlist[r1i]
            if prod in reviewtime[r1]:
                for r2i in range(r1i + 1, len(userlist)):
                    r2 = userlist[r2i]
                    if prod in reviewtime[r2]:
                        avg = avg + cosine(reviewcontent[r1][prod],
                                           reviewcontent[r2][prod])
                        c = c + 1
        if c != 0:
            avg = avg / (c * 1.0)
        if avg > maxx:
            maxx = avg
    return maxx


def GMCS(group):

    avg = 0
    totc = len(group.users)

    if len(group.prods) <= 1:
        return 0
    for user in group.users:
        ans = 0
        c = 0
        prodlist = list(group.prods)
        for p1i in range(len(prodlist)):
            p1 = prodlist[p1i]
            if p1 in reviewtime[user]:
                for p2i in range(p1i + 1, len(prodlist)):
                    p2 = prodlist[p2i]

                    if p2 in reviewtime[user]:
                        ans = ans + cosine(reviewcontent[user][p1],
                                           reviewcontent[user][p2])
                        c = c + 1
        if c != 0:
            avg = avg + (ans * 1.0) / c
        else:
            totc = totc - 1
    if totc == 0:
        return 0
    return (avg * 1.0) / (totc)


def calc_score(g, Lsub):
    score = []
    ans = averagetimewindow_ratingvariance(g, Lsub)
    score = [
        reviewtightness(g, Lsub),
        neighbortightness(g, Lsub),
        producttightness(g), ans[0], ans[1],
        productreviewerratio(g)
    ]
    return (score, sum(score) / (len(score) * 1.0), groupsize(g), GCS(g))



def create_groups():
    finalgrps={}
    with open("/content/drive/My Drive/convertcsv.json", 'r') as fp:
        finalgrps = json.load(fp)
        x=0
        v=5
        for grp in finalgrps:
            '''
            finalgrps[grp]['users']=map(int, finalgrps[grp]['users'])
            
            finalgrps[grp]['prods']=map(int, finalgrps[grp]['prods'])

            k=list(finalgrps[grp]['users'])
            
            finalgrps[grp]['users']=k
            
            k=list(finalgrps[grp]['prods'])
            finalgrps[grp]['prods']=k
            '''
            if len(finalgrps[grp]['users'])>1:

                group=Groups(finalgrps[grp]['users'],finalgrps[grp]['prods'])

                Lsub=1.0/(1+(math.exp(3-len(group.users)-len(group.prods))))
                ans=calc_score(group,Lsub)
                scorepred=ans[0]
                spamicity=ans[1]
                gs = ans[2]
                rcs = ans[3]

                c=0
                denom=0
                for u in group.users:
                    if u in fake:
                        c=c+1
                    denom=denom+1
                store=(c*1.0)/denom

                c = 0
                denom = 0
                for u in group.users:
                    for p in group.prods:
                        if p in wholerev[u]:
                            if int(wholerev[u][p].label) == -1:
                                c = c + 1
                            denom = denom + 1
                if len(group.prods) == 0:
                    denom = 1
                    c = 0
                
                
                if x not in grps:
                    grps[x]={'id':x,'users':list(group.users),'prods':list(group.prods),'scorepred':scorepred, 'scoregt':store, 'scoregtreviewprec':(c*1.0)/denom, 'fakegt':0,'fakepred':spamicity,'gs': gs,'rcs': rcs}
                    x=x+1

        tc=0
        for grp in grps: 
            scorepred=grps[grp]['scorepred']          
            summ=sum(scorepred[:v])/6.0
            if summ>0.4:
                tc=tc+1

        if tc<100:
            v=v+1

        for grp in grps:
            scorepred=grps[grp]['scorepred']     
            summ=sum(scorepred[:v])/6.0
            if summ>0.4:
                grps2[len(grps2)]=grps[grp]
                grps2[len(grps2)-1]['id']=len(grps2)-1





In [ ]:
grps={}
grps2={}
create_groups()

with open("/content/drive/My Drive/refinedconvertcsv (1).json", 'w') as fp:
    json.dump(grps2, fp,indent=True)  
print ('end')


end
end


In [ ]:
theta = 0.4
MP = 10000
tau = 30.0
suspicion = {}
adj = {}
ans = {}
refinedgroups = []
fileedgelist = open("/content/drive/My Drive/edgelist.txt", 'w')
fileadjlist = open("/content/drive/My Drive/adjlist.txt", 'w')
for prod in allprods:
    if prod not in suspicion:
        suspicion[prod] = 1
        suspicion[prod] = (2.0 / (1 + math.exp(-1 *(pow(abs(MP - len(allprods[prod])), theta)) + pow(2, theta)))) - 1

        userlist = set()
        for u in allprods[prod]:
            userlist.add(u)
            # userlist.add(u.userid)
        userlist = list(userlist)
        for u1i in range(len(userlist)):
            u1 = userlist[u1i]
            if u1 not in ans:
                ans[u1] = {}
            for u2i in range(u1i + 1, len(userlist)):
                u2 = userlist[u2i]
                if u2 not in ans[u1]:
                    ans[u1][u2] = 0
                if abs(reviewtime[u1][prod] - reviewtime[u2][prod]) <= tau and abs(reviewrating[u1][prod] -reviewrating[u2][prod]) < 2:
                    ans[u1][u2] = ans[u1][u2] + 0.3 * suspicion[prod] * ((1 -(abs(reviewtime[u1][prod] - reviewtime[u2][prod])) /tau) +(1 -(abs(reviewrating[u1][prod] - reviewrating[u2][prod]))/ 2.0) + 0.4 * cosine(reviewcontent[u1][prod],reviewcontent[u2][prod]))

    for u1 in ans:
        if u1 not in adj:
            adj[u1] = set()
        for u2 in ans[u1]:
            if u2 not in adj:
                adj[u2] = set()
            jaccard = (1.0 * len(set(allusers[u1]).intersection(set(allusers[u2])))) / (len(set(allusers[u1]).union(set(allusers[u2]))))
            ans[u1][u2] = ans[u1][u2] * jaccard
            similarity = (2.0 / (1 + math.exp(-1 * ans[u1][u2]))) - 1
            if similarity > 0:
                fileedgelist.write(str(u1) + "\t" + str(u2) + "\t" + str(similarity) + "\n")
                adj[u1].add(u2)
                adj[u2].add(u1)

    for u1 in adj:
        fileadjlist.write(str(u1))
        for u2 in adj[u1]:
            fileadjlist.write("\t" + str(u2))
        fileadjlist.write("\n")

    

In [ ]:
!pip install node2vec

In [ ]:
import node2vec
from node2vec import Node2Vec
from gensim.models import Word2Vec

In [ ]:
G=nx.read_weighted_edgelist("C:\\Users\\Administrator\\Desktop\\minor pro\\edgelist.txt",create_using=nx.DiGraph())
node2vec = Node2Vec(G,dimensions=128, walk_length=80, num_walks=10, workers=8) 
model = node2vec.fit(window=10, min_count=1, batch_words=4)
model.wv.save_word2vec_format("C:\\Users\\Administrator\\Desktop\\minor pro\\embeddings.emd")

In [ ]:
from scipy.spatial import distance
import operator
import json
import numpy as np
import sys
import math


class Groups:
    def __init__(self,users,prods,score,scoregt,id):
        self.users=users
        self.prods=prods
        self.score=score
        self.scoregt=scoregt
        self.id=id

    def __lt__(self, other):
        return len(self.users) < len(other.users)

c=0
groups=set()
grps={}
grpmapping={}
avggrpsize=0
size=0
with open( "/content/drive/My Drive/refinedconvertcsv (1).json", 'r') as fp:
    finalgrps = json.load(fp)

filee=open("/content/drive/My Drive/embedding.emd",'r')
mapping={}
c=0
for f in filee:
    c=c+1
    if c==1:
        continue
    fsplit=f.strip().split(" ")
    print(fsplit[1])
    if fsplit[0] not in mapping:
        mapping[int(fsplit[0])]=map(float, fsplit[1:])
        k=list(mapping[int(fsplit[0])])
        mapping[int(fsplit[0])]=k
        emb_size=len(fsplit[1:])
filee.close()


userset=set()
gtscore={}
size={}
grpusers={}
for g1 in finalgrps:
    '''
    finalgrps[g1]['users']=map(int,finalgrps[g1]['users'])
    finalgrps[g1]['prods']=map(int,finalgrps[g1]['prods'])
    '''
    group=Groups(finalgrps[g1]['users'],finalgrps[g1]['prods'],finalgrps[g1]['fakegt'],finalgrps[g1]['scoregt'],finalgrps[g1]['id'])
    if len(finalgrps[g1]['users']) not in size:
        size[len(finalgrps[g1]['users'])]=0
    size[len(finalgrps[g1]['users'])]=size[len(finalgrps[g1]['users'])]+1

    groups.add(group)

    grpmapping[finalgrps[g1]['id']]=group
    if finalgrps[g1]['id'] not in grpusers:
        grpusers[finalgrps[g1]['id']]=finalgrps[g1]['users']

    avggrpsize=avggrpsize+len(finalgrps[g1]['users'])


r_gt=[]
avggrpsize=avggrpsize/(len(groups)*1.0)
score={}

def density():
    for gm in grpmapping:
        g=grpmapping[gm]
        avg=[0 for i in range(emb_size)]
        ans=0
        for u in g.users:
            if u in mapping:
                avg=[avg[i]+mapping[u][i] for i in range(emb_size)]

        avg=[(a*1.0)/len(g.users) for a in avg]
        for u in g.users:
            if u in mapping:
                ans=ans+distance.euclidean(mapping[u], avg)
        if gm not in score:
            score[gm]=ans/(1.0*len(g.users))
    sorted_score=sorted(score.items(), key=operator.itemgetter(1))
    return sorted_score

def rank():
    sorted_score=density()
    filew=open("/content/drive/My Drive/ranking.json",'w')
    for grp in sorted_score:
        filew.write(str(grp[0])+"\n")
    filew.close()

rank()
print ('end')

In [ ]:
'''
ds={}
for i in allprods:
  ds[i]=[]
  for j in range(len(allprods[i])):
    ds[i].append(wholerev[allprods[i][j]][i])
  ds[i].sort(key=lambda x: x.date)
  for j in range(len(allprods[i])):
    ds[i][j]=wholerev[allprods[i][j]][i].userid
print(len(ds))
'''

In [ ]:
'''
import math
h=0
temp={}



for i in ds:
    degk=len(ds[i])
    sp=2/((1+math.exp(-pow((10000-degk),0.4))+pow(2,0.4)))
    sp=sp-1
    for j in range(len(ds[i])):
        for k in range(j+1,len(ds[i])):
            var1=reviewtime[ds[i][j]][i]
            var2=reviewtime[ds[i][k]][i]
            rat1=reviewrating[ds[i][j]][i]
            rat2=reviewrating[ds[i][k]][i]
            res1=(abs(var1-var2)/20)*0.3
            res2=(abs(rat1-rat2)/2)*0.3
            if((var1-var2)>20):
                result=0
                break
            elif (rat1-rat2)>2:
                result=0
            else:
                res3=cosine(reviewcontent[ds[i][j]][i], reviewcontent[ds[i][k]][i])
                res3=res3*0.4
                result=sp*(res1+res2+res3)
            temp[h]={}
            temp[h][0]=ds[i][j]
            temp[h][1]=ds[i][k]
            temp[h][2]=i
            temp[h][3]=result
            h=h+1

d={}

for i in range(len(temp)):
    if temp[i][0] not in d:
        d[temp[i][0]]={}
        d[temp[i][0]][temp[i][1]]=temp[i][3]
    else: 
        if temp[i][1] not in d[temp[i][0]]:
            d[temp[i][0]][temp[i][1]]=temp[i][3]
        else:
            d[temp[i][0]][temp[i][1]]=d[temp[i][0]][temp[i][1]]+temp[i][3]
'''



In [ ]:
'''re={}
for i in d:
  re[i]={}
  s=set(allusers[i])
  for j in d[i]:
    p=set(allusers[j])
    union=s.union(p)
    inter=s.intersection(p)
    jaccard=len(inter)/len(union)*1.0
    re[i][j]=d[i][j]*jaccard'''

In [ ]:
'''
import networkx as nx
import matplotlib.pyplot as plt




Gt=nx.Graph()
for i in re:
    for j in re[i]:
            if(re[i][j]!=0):
                Gt.add_edge(i,j,weight=re[i][j])
                
pos=nx.shell_layout(Gt) # positions for all nodes

# nodes
nx.draw_networkx_nodes(Gt,pos,node_size=600,alpha=0.3)

# edges
nx.draw_networkx_edges(Gt,pos,edgelist=Gt.edges(),width=3)

nx.draw_networkx_labels(Gt,pos,font_size=10,font_family='sans-serif')

 


nx.write_gml(Gt, "/content/drive/My Drive/Graph.gml")



'''

In [ ]:
'''import networkx as nx
from node2vec import Node2Vec
Gt= nx.read_gml("/content/drive/My Drive/Graph.gml")
node2vec = Node2Vec(Gt,num_walks=1) '''

In [ ]:
'''model = node2vec.fit(window=10, min_count=1, batch_words=4)
model.wv.save_word2vec_format("/content/drive/My Drive/embedding")'''

In [ ]:
'''import json
import collections
import numpy as np
import matplotlib.pyplot as plot
with open( "/content/drive/My Drive/refinedconvertcsv (1).json", 'r') as fp:
    finalgrps = json.load(fp)

a=[]
for g1 in finalgrps:
    a.append(len(finalgrps[g1]['users']))
    
k=collections.Counter(a)  

b=[]
c=[]
for key,value in k.items():
    b.append(value)
    c.append(key)
print(b)
c.sort()
y_pos = np.arange(len(c))
    
plot.figure(figsize=(70, 3))
plot.bar(y_pos, b, width=0.6 ,align='edge',alpha=0.8)
plot.xticks(y_pos,c)
plot.xlabel('Group size')
plot.ylabel('Number of groups')
plot.title('Groups on YelpNYC')


plt.savefig('barchart.png')'''

In [ ]:
import pandas as pd

# read data
reviews_df = pd.read_excel("/content/drive/My Drive/Yelp dataset.xlsx")
# append the positive and negative text reviews
reviews_df["review"] = reviews_df["Review_Text"]
# create the label


In [ ]:
# remove 'No Negative' or 'No Positive' from text
reviews_df["review"] = reviews_df["review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
# add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews_df["sentiments"] = reviews_df["review"].apply(lambda x: sid.polarity_scores(x))
reviews_df = pd.concat([reviews_df.drop(['sentiments'], axis=1), reviews_df['sentiments'].apply(pd.Series)], axis=1)

In [ ]:
# storing final sentimental scores of each review in a  new file output.xlsx
reviews_df.to_excel("/content/drive/My Drive/output.xlsx")

In [ ]:
import pandas as pd
new_df = pd.read_excel("/content/drive/My Drive/output.xlsx")

In [ ]:
js = pd.read_json("/content/drive/My Drive/convertcsv.json")
# Book1 is an empty file initially
book= pd.read_excel("/content/drive/My Drive/Book1.xlsx")
meta=pd.read_excel("/content/drive/My Drive/Yelp Metadata.xlsx")
# Adding rating column from Metadata.xlsx to output.xlsx
z=meta["Rating"]
new_df['Rating']=z
#inner join on review id
new=pd.merge(meta,new_df,how='inner',on=['Review_id'])
# retreiving total no of generated groups in convertcsv.json
size1=js.shape[1]

In [ ]:
import numpy as np
# for loop for each group
for i in range (size1):
  # list of reveiwers in a group 
  a=js.iloc[1,i]
  #list of products spammed by a group
  b=js.iloc[2,i]
 # book['Review_id'] = []
 #creating an empty file for calculation purpose
  book= pd.read_excel("/content/drive/My Drive/Book1.xlsx")
  book['Review_id'] = a
  #inner join on review_id
  c=pd.merge(book,new_df,how='inner',on=['Review_id'])
  bok= pd.read_excel("/content/drive/My Drive/Book2.xlsx")
  bok['Product_id']=b
  #total no of products spammed by a group
  size=bok.shape[0]
  #inner join on product_id
  d=pd.merge(bok,c,how='inner',on=['Product_id'])
  #calculation of average rating and and average sentimental score on each product by a group
  d.groupby('Product_id').transform('first')
  #storing average of sentimental scores for each product by  group id i
  e=d.groupby('Product_id')['compound'].mean()
  d.Rating=d.Rating.astype('float64') 
  #storing average of rating for each product by  group id i
  f=d.groupby('Product_id')['Rating'].mean()
  y=float(f.iloc[0])
  g=f/5
  #(average of sentimental scores for each product by  group id i +average of rating for each product by  group id i)/2
  fin=(e+g)/2
  k=[]
  m=[]
  #Labeling each product  as positive and negative in accordance with the group
  #storing final score for each product in list k
  for t in range (size):
    k.append(fin.iloc[t])
  #storing label for each product in list m
  for t in range (size):
    if(k[t]>=0.5):
      m.append('POS')
    else:
      m.append('NEG')
  # creating label column for each group
  js.loc["Label",i]=m
  #book.drop(["Review_id"], axis=1,inplace=True)
  del book
js


In [ ]:
#putting final result into file groups.json
js.to_json("/content/drive/My Drive/groups.json")

In [ ]:
#part2
import pandas  as pd
df = pd.read_json('/content/drive/My Drive/groups.json')
df_transpose = df.transpose().drop(['Label'], axis=1)
df_transpose.rename(columns={'users':'fraud_group','id':'fraud_group_id','prods':'Product_id'}, inplace = True)
df_transpose = df_transpose.explode('Product_id')
df_transpose

,fraud_group_id,fraud_group,Product_id,scorepred,scoregt,fakegt,fakepred
0,0,"[1028, 1031, 1037, 1048, 923, 924, 939, 1082, ...",0,0,0,0,0
1,1,"[1027, 1029, 1030, 1032, 1033, 1035, 1039, 104...",0,0,0,0,0
2,2,"[1038, 1045, 1046, 1055, 1056, 932, 933, 934, ...",0,0,0,0,0
3,3,"[1024, 1025, 962, 1026, 1057, 1058, 1061, 969,...",0,0,0,0,0
4,4,"[1016, 972]",0,0,0,0,0
...,...,...,...,...,...,...,...
488,488,"[8203, 18334]",63,0,0,0,0
489,489,"[20640, 20490, 20492, 20493, 20494, 20495, 204...",72,0,0,0,0
489,489,"[20640, 20490, 20492, 20493, 20494, 20495, 204...",95,0,0,0,0
490,490,"[30933, 30942]",100,0,0,0,0


In [ ]:
df_transpose = df_transpose.drop("scorepred", axis=1)
df_transpose = df_transpose.drop("scoregt", axis=1)
df_transpose = df_transpose.drop("fakegt", axis=1)
df_transpose = df_transpose.drop("fakepred", axis=1)
df_transpose

,fraud_group_id,fraud_group,Product_id
0,0,"[1028, 1031, 1037, 1048, 923, 924, 939, 1082, ...",0
1,1,"[1027, 1029, 1030, 1032, 1033, 1035, 1039, 104...",0
2,2,"[1038, 1045, 1046, 1055, 1056, 932, 933, 934, ...",0
3,3,"[1024, 1025, 962, 1026, 1057, 1058, 1061, 969,...",0
4,4,"[1016, 972]",0
...,...,...,...
488,488,"[8203, 18334]",63
489,489,"[20640, 20490, 20492, 20493, 20494, 20495, 204...",72
489,489,"[20640, 20490, 20492, 20493, 20494, 20495, 204...",95
490,490,"[30933, 30942]",100


In [ ]:
df_transpose_without_fraud_group = df_transpose.drop("fraud_group", axis=1)
df_transpose_without_fraud_group

,fraud_group_id,Product_id
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
488,488,63
489,489,72
489,489,95
490,490,100


In [ ]:
df_explode = df_transpose.explode('fraud_group')
df_explode.rename(columns={'fraud_group':'Review_id'}, inplace = True)
df_explode_without_prod = df_explode.drop("Product_id", axis=1)
df_explode

,fraud_group_id,Review_id,Product_id
0,0,1028,0
0,0,1031,0
0,0,1037,0
0,0,1048,0
0,0,923,0
...,...,...,...
490,490,30942,100
490,490,30933,95
490,490,30942,95
490,490,30933,95


In [ ]:
df_explode1 = df_explode.drop("fraud_group_id", axis=1)
df_explode1

,Review_id,Product_id
0,1028,0
0,1031,0
0,1037,0
0,1048,0
0,923,0
...,...,...
490,30942,100
490,30933,95
490,30942,95
490,30933,95


In [ ]:
new_df = pd.read_excel('/content/drive/My Drive/Yelp Metadata.xlsx')

In [ ]:
#new_df = new_df.drop("Label", axis=1)
new_df = new_df.drop("Review_Date", axis=1)
new_df

,Review_id,Product_id,Rating
0,923,0,3
1,924,0,3
2,925,0,4
3,926,0,4
4,927,0,4
...,...,...,...
359047,161146,349,5
359048,116424,349,5
359049,161147,349,5
359050,97930,349,5


In [ ]:
new_df1 = pd.read_excel('/content/drive/My Drive/Yelp Metadata.xlsx')

In [ ]:
new_df1 = new_df1.drop("Label", axis=1)
new_df1 = new_df1.drop("Review_Date", axis=1)
new_df1 = new_df1.drop("Rating", axis=1)
new_df1

,Review_id,Product_id
0,923,0
1,924,0
2,925,0
3,926,0
4,927,0
...,...,...
359047,161146,349
359048,116424,349
359049,161147,349
359050,97930,349


In [ ]:
tot_rat_sum = new_df.groupby('Product_id')['Rating'].sum().reset_index()
tot_reviewer_count = new_df.groupby('Product_id')['Review_id'].count().reset_index()
tot_reviewer_count.rename(columns={'Review_id':'Total_reviewer_count'}, inplace = True)
total_df1 = pd.merge(tot_rat_sum,tot_reviewer_count, how = 'inner', on = ['Product_id'])
total_df1['part_one'] = total_df1['Rating']/total_df1['Total_reviewer_count']
total_df1
#tot_rat_sum

,Product_id,Rating,Total_reviewer_count,part_one
0,0,842,210,4.009524
1,1,1095,253,4.328063
2,2,638,166,3.843373
3,3,246,59,4.169492
4,4,2206,547,4.032907
...,...,...,...,...
918,918,1455,389,3.740360
919,919,2047,486,4.211934
920,920,1253,306,4.094771
921,921,801,192,4.171875


In [ ]:
df_prep = new_df1.merge(df_explode1, indicator=True, how = "left")[lambda x: x._merge=='left_only'].drop('_merge',1)
#df_prep = pd.concat([new_df1,df_explode1]).drop_duplicates(keep=False)
#df_prep
orig_df1 = pd.merge(new_df,df_prep, how = 'inner', on = ['Review_id','Product_id'])
#Review_id is Count of doriginal reviewers
#orig_df1
orig_rat_sum = orig_df1.groupby('Product_id')['Rating'].sum().reset_index()
orig_rat_count = orig_df1.groupby('Product_id')['Review_id'].count().reset_index()
final_df = pd.merge(orig_rat_sum,orig_rat_count, how = 'inner', on = ['Product_id'])
final_df['part_two'] = final_df['Rating']/final_df['Review_id']
final_df

,Product_id,Rating,Review_id,part_two
0,2,638,166,3.843373
1,4,180,60,3.000000
2,7,345,115,3.000000
3,11,2,1,2.000000
4,12,630,169,3.727811
...,...,...,...,...
835,918,1455,389,3.740360
836,919,2047,486,4.211934
837,920,1253,306,4.094771
838,921,801,192,4.171875


In [ ]:
import numpy as np
df_test = pd.merge(total_df1,final_df, how = 'left', on = ['Product_id'])
df_test = df_test.replace(np.nan,0)
df_test

,Product_id,Rating_x,Total_reviewer_count,part_one,Rating_y,Review_id,part_two
0,0,842,210,4.009524,0.0,0.0,0.000000
1,1,1095,253,4.328063,0.0,0.0,0.000000
2,2,638,166,3.843373,638.0,166.0,3.843373
3,3,246,59,4.169492,0.0,0.0,0.000000
4,4,2206,547,4.032907,180.0,60.0,3.000000
...,...,...,...,...,...,...,...
918,918,1455,389,3.740360,1455.0,389.0,3.740360
919,919,2047,486,4.211934,2047.0,486.0,4.211934
920,920,1253,306,4.094771,1253.0,306.0,4.094771
921,921,801,192,4.171875,801.0,192.0,4.171875


In [ ]:
#final_df1 = pd.merge(final_df,total_df1, how = 'inner', on = ['Product_id'])
#final_df1
df_test["diff_p1_p2"] = (df_test["part_one"] - df_test["part_two"])
df_test

,Product_id,Rating_x,Total_reviewer_count,part_one,Rating_y,Review_id,part_two,diff_p1_p2
0,0,842,210,4.009524,0.0,0.0,0.000000,4.009524
1,1,1095,253,4.328063,0.0,0.0,0.000000,4.328063
2,2,638,166,3.843373,638.0,166.0,3.843373,0.000000
3,3,246,59,4.169492,0.0,0.0,0.000000,4.169492
4,4,2206,547,4.032907,180.0,60.0,3.000000,1.032907
...,...,...,...,...,...,...,...,...
918,918,1455,389,3.740360,1455.0,389.0,3.740360,0.000000
919,919,2047,486,4.211934,2047.0,486.0,4.211934,0.000000
920,920,1253,306,4.094771,1253.0,306.0,4.094771,0.000000
921,921,801,192,4.171875,801.0,192.0,4.171875,0.000000


In [ ]:
final_df1 = df_test
final_df1 = final_df1.drop("Rating_x", axis=1)
final_df1 = final_df1.drop("Rating_y", axis=1)
final_df1 = final_df1.drop("Review_id", axis=1)
final_df1 = final_df1.drop("Total_reviewer_count", axis=1)
final_df1 = final_df1.drop("part_one", axis=1)
final_df1 

,Product_id,part_two,diff_p1_p2
0,0,0.000000,4.009524
1,1,0.000000,4.328063
2,2,3.843373,0.000000
3,3,0.000000,4.169492
4,4,3.000000,1.032907
...,...,...,...
918,918,3.740360,0.000000
919,919,4.211934,0.000000
920,920,4.094771,0.000000
921,921,4.171875,0.000000


In [ ]:
final_df1.to_json("/content/drive/My Drive/Minor_Part1_res.json")

In [ ]:
fraud_group_avg = pd.merge(new_df,df_explode, how = 'inner', on = ['Review_id','Product_id'])
fraud_group_rat_sum = fraud_group_avg.groupby('fraud_group_id')['Rating'].sum().reset_index()
fraud_group_count = fraud_group_avg.groupby('fraud_group_id')['Review_id'].count().reset_index()
fraud_group_avg_final = pd.merge(fraud_group_rat_sum,fraud_group_count, how = 'inner', on = ['fraud_group_id'])
fraud_group_avg_final.rename(columns={'Review_id':'Reviewer_count'}, inplace = True)
fraud_group_avg_final['Each_group_avg_rat'] = fraud_group_avg_final['Rating']/fraud_group_avg_final['Reviewer_count']
fraud_group_avg_final.rename(columns={'Reviewer_count':'Fraud_grp_Reviewer_count','Rating':'Fraud_group_rat_sum'}, inplace = True)
fraud_group_avg_final
#fraud_group_count
#fraud_group_rat_sum
#fraud_group_avg
#average rating of each group

In [ ]:
total_df1.rename(columns={'Rating':'Total_rating'}, inplace = True)
total_df1

,Product_id,Total_rating,Total_reviewer_count,part_one
0,0,842,210,4.009524
1,1,1095,253,4.328063
2,2,638,166,3.843373
3,3,246,59,4.169492
4,4,2206,547,4.032907
...,...,...,...,...
918,918,1455,389,3.740360
919,919,2047,486,4.211934
920,920,1253,306,4.094771
921,921,801,192,4.171875


In [ ]:
#df_transpose_without_fraud_group
wholesome_df = pd.merge(df_transpose_without_fraud_group,fraud_group_avg_final, how = 'inner', on = ['fraud_group_id'])
wholesome_df1 = pd.merge(total_df1,wholesome_df, how = 'inner', on = ['Product_id'])
wholesome_df1

,Product_id,Total_rating,Total_reviewer_count,part_one,fraud_group_id,Fraud_group_rat_sum,Fraud_grp_Reviewer_count,Each_group_avg_rat
0,0,842,210,4.009524,0,69,23,3.0
1,0,842,210,4.009524,1,424,106,4.0
2,0,842,210,4.009524,2,315,63,5.0
3,0,842,210,4.009524,3,32,16,2.0
4,0,842,210,4.009524,4,2,2,1.0
...,...,...,...,...,...,...,...,...
494,884,358,86,4.162791,252,96,24,4.0
495,884,358,86,4.162791,253,220,44,5.0
496,900,142,30,4.733333,357,120,24,5.0
497,900,142,30,4.733333,358,16,4,4.0


In [ ]:
wholesome_df1["Rating_Difference"] = (wholesome_df1["Total_rating"] - wholesome_df1["Fraud_group_rat_sum"])
wholesome_df1["Count_difference"] = (wholesome_df1["Total_reviewer_count"] - wholesome_df1["Fraud_grp_Reviewer_count"])
wholesome_df1["Each_grp_impact_avg"] = (wholesome_df1["Rating_Difference"]/wholesome_df1["Count_difference"])
wholesome_df1

,Product_id,Total_rating,Total_reviewer_count,part_one,fraud_group_id,Fraud_group_rat_sum,Fraud_grp_Reviewer_count,Each_group_avg_rat,Rating_Difference,Count_difference,Each_grp_impact_avg
0,0,842,210,4.009524,0,69,23,3.0,773,187,4.133690
1,0,842,210,4.009524,1,424,106,4.0,418,104,4.019231
2,0,842,210,4.009524,2,315,63,5.0,527,147,3.585034
3,0,842,210,4.009524,3,32,16,2.0,810,194,4.175258
4,0,842,210,4.009524,4,2,2,1.0,840,208,4.038462
...,...,...,...,...,...,...,...,...,...,...,...
494,884,358,86,4.162791,252,96,24,4.0,262,62,4.225806
495,884,358,86,4.162791,253,220,44,5.0,138,42,3.285714
496,900,142,30,4.733333,357,120,24,5.0,22,6,3.666667
497,900,142,30,4.733333,358,16,4,4.0,126,26,4.846154


In [ ]:
wholesome_df3 = pd.merge(wholesome_df1,final_df1, how = 'left', on = ['Product_id'])
#wholesome_df3 = wholesome_df3.drop("Total_rating", axis=1)
#wholesome_df3 = wholesome_df3.drop("Total_reviewer_count_x", axis=1)
#wholesome_df3 = wholesome_df3.drop("part_one_x", axis=1)
#wholesome_df3 = wholesome_df3.drop("part_one_y", axis=1)
#wholesome_df3 = wholesome_df3.drop("fraud_group_id", axis=1)
#wholesome_df3 = wholesome_df3.drop("Fraud_group_rat_sum", axis=1)
#wholesome_df3 = wholesome_df3.drop("Fraud_grp_Reviewer_count", axis=1)
#wholesome_df3 = wholesome_df3.drop("Each_group_avg_rat", axis=1)
#wholesome_df3 = wholesome_df3.drop("Rating_Difference", axis=1)
#wholesome_df3 = wholesome_df3.drop("Count_difference", axis=1)
#wholesome_df3 = wholesome_df3.drop("Each_grp_impact_avg", axis=1)
#wholesome_df3 = wholesome_df3.drop("Total_reviewer_count_y", axis=1)
#wholesome_df3 = wholesome_df3.drop("Rating_x", axis=1)
#wholesome_df3 = wholesome_df3.drop("Rating_y", axis=1)
#wholesome_df3 = wholesome_df3.drop("Review_id", axis=1)
wholesome_df3

,Product_id,Total_rating,Total_reviewer_count,part_one,fraud_group_id,Fraud_group_rat_sum,Fraud_grp_Reviewer_count,Each_group_avg_rat,Rating_Difference,Count_difference,Each_grp_impact_avg,part_two,diff_p1_p2
0,0,842,210,4.009524,0,69,23,3.0,773,187,4.133690,0.0,4.009524
1,0,842,210,4.009524,1,424,106,4.0,418,104,4.019231,0.0,4.009524
2,0,842,210,4.009524,2,315,63,5.0,527,147,3.585034,0.0,4.009524
3,0,842,210,4.009524,3,32,16,2.0,810,194,4.175258,0.0,4.009524
4,0,842,210,4.009524,4,2,2,1.0,840,208,4.038462,0.0,4.009524
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,884,358,86,4.162791,252,96,24,4.0,262,62,4.225806,0.0,4.162791
495,884,358,86,4.162791,253,220,44,5.0,138,42,3.285714,0.0,4.162791
496,900,142,30,4.733333,357,120,24,5.0,22,6,3.666667,0.0,4.733333
497,900,142,30,4.733333,358,16,4,4.0,126,26,4.846154,0.0,4.733333


In [ ]:
wholesome_df3.to_json("/content/drive/My Drive/new_result_temp2.jso

In [ ]:
wholesome_df3["First_difference"] =( wholesome_df3["Each_grp_impact_avg"] - wholesome_df3["diff_p1_p2"] )
wholesome_df3

In [ ]:
wholesome_df3

In [ ]:
wholesome_df3.to_json("/content/drive/My Drive/Minor_res_part2.json")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#myplot = final_df1.plot(final_df1['Product_id'], kind ='bar', legend = None)
lines = final_df1.plot.line(x='Product_id', y='diff_p1_p2')
plt.plot()
plt.savefig('/content/drive/My Drive/final_img_res.png', dpi=300)